# ADNI Alzheimer's Neuroiamging Dataset Handling

In [11]:
import os
import shutil
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


We shall unnest the files by gathering each of the file paths as a flattened list.

In [12]:
def join_listdir(a, bs):
    return [os.path.join(a, b) for b in bs]

def list_listdir(dirs, paths=[]):
    return [
        paths + list_listdir(join_listdir(item, os.listdir(item)), paths) if os.path.isdir(item)
        else paths + [item]
        for item in dirs
    ]

def flatten(xs):
    if len(xs) == 0:
        return xs
    if isinstance(xs[0], list):
        return flatten(xs[0]) + flatten(xs[1:])
    return xs[:1] + flatten(xs[1:])

data_dir = "/Data/adni_frmi_alzheimers/data/ADNI_original"
paths = list_listdir([data_dir])
paths = flatten(paths)

Now for each path we can copy to the new target destination.

In [40]:
def get_target_path(path):
    fmri_nii_dir = "/Data/adni_frmi_alzheimers/data/ADNI/fmri_nii"
    meta_xml_dir = "/Data/adni_frmi_alzheimers/data/ADNI/meta_xml"
    filename = os.path.basename(path)
    if filename[-4:] == '.nii':
        return os.path.join(fmri_nii_dir, filename)
    elif filename[-4:] == '.xml':
        return os.path.join(meta_xml_dir, filename)
    print(f"ERROR: unrecognised file type (.{filename.split('.')[1]})")

for path in paths:
    target_path = get_target_path(path)
    shutil.copyfile(path, target_path)

As the fMRIs are .nii volumes we will need to convert them to a standard 2D image for training.

For now we shall just take the middle slice of each volume.